In [1]:
#Neural Network using Pytorch

#Neural networks can be constructed using the torch.nn package.

#A typical training procedure for a neural network is as follows:

# - Define the neural network that has some learnable parameters (or weights)
# - Iterate over a dataset of inputs
# - Process input through the network
# - Compute the loss (how far is the output from being correct)
# - Propagate gradients back into the network’s parameters
# - Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

In [14]:
#Define the network

import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #1 input image channel, 6 output channel and 5x5 square convolution kernel
        self.conv1= nn.Conv2d(1,6,5)
        self.conv2= nn.Conv2d(6, 16, 5)
        #An affine operation: y= Wx + b
        self.fc1= nn.Linear(16*5*5, 120)
        self.fc2= nn.Linear(120, 84)
        self.fc3= nn.Linear(84, 10)
        
    def forward(self, x):
        #Max Pooling over a 2x2 window
        x= F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        #If the size is a square, we can specify a single number
        x= F.max_pool2d(F.relu(self.conv2(x)), 2)
        x= x.view(-1, self.num_flat_features(x))
        x= x.view(-1, self.num_flat_features(x))
        x= F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x= self.fc3(x)
        
        return x
        
    
    def num_flat_features(slef, x):
        size= x.size()[1:] #All teh sizes except the batch size
        num_features= 1
        for s in size:
            num_features*=s
        
        return num_features
    
    
net= Net()
print(net)
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [16]:
#The learnable parameters of a model are returned by net.parameters()

params= list(net.parameters())
print(len(params))
print(params[0].size()) #conv's weight

10
torch.Size([6, 1, 5, 5])


In [17]:
input= torch.randn(1,1,32,32)
out= net(input)
print(out)

tensor([[ 0.1153,  0.0577,  0.0137, -0.0867,  0.0495,  0.0708,  0.0764,  0.1047,
         -0.1087, -0.0213]], grad_fn=<ThAddmmBackward>)


In [18]:
#Zero the gradient buffers of all parameters and backprops with random gradients:

net.zero_grad()
out.backward(torch.randn(1,10))

In [19]:
#Loss Function

output= net(input)
target= torch.randn(10) #a dummy target for example
target= target.view(1,-1) #make it of the shape of output
criterion= nn.MSELoss()

loss= criterion(output, target)
print(loss)

tensor(1.0864, grad_fn=<MseLossBackward>)


In [20]:
#graph of computations that looks like this:

#input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
#      -> view -> linear -> relu -> linear -> relu -> linear
#      -> MSELoss
#      -> loss

In [21]:
print(loss.grad_fn) #MSE Loss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

In [22]:
#BackProp

net.zero_grad() #zeros the gradient buffers for all the parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0184,  0.0106,  0.0025, -0.0053, -0.0095, -0.0037])


In [23]:
#Update the weights
learning_rate=0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

In [24]:
import torch.optim as optim

#Create optimizer
optimizer= optim.SGD(net.parameters(), lr=0.01)

#in training loop
optimizer.zero_grad() #zero the gradient buffers
output= net(input)
loss= criterion(output, target)
loss.backward()
optimizer.step() #Does the update
